In [144]:
import copy
from time import time
from random import randint
from random import random
from collections import defaultdict

#Input
pos_list = ['.','.','.','.','.','.','.','.','.','.','.']
stacks = {2:['C','C'], 4:['D','B'], 6:['A','A'], 8:['B','D']}
costs = {'A':1, 'B':10, 'C':100, 'D':1000}
types = {'A':2,'B':4,'C':6,'D':8}
types_rev = {2:'A',4:'B',6:'C',8:'D'}

#Program
def visible_pos(nstack, pos_list):
    """Returns the list of possible positions from a given stack"""
    vis_pos=[]
    for i in range(len(pos_list)):
        if i<nstack and i not in [2,4,6,8] and set(pos_list[i:nstack+1])=={'.'}:
            vis_pos.append(i)
        if i>nstack and i not in [2,4,6,8] and set(pos_list[nstack:i+1]) == {'.'}:
            vis_pos.append(i)
    return vis_pos
    
def visible_stacks(pos, pos_list, stacks):
    """Says if amphipod at position pos can go to its stack"""
    x = pos_list[pos]
    i = types[x]
    if len(stacks[i])==2 or (len(stacks[i])==1 and stacks[i][0] != x):
        return False
    if pos==i-1 or (pos<i and set(pos_list[pos+1:i]) == {'.'}):
        return True
    elif pos== i+1 or (i<pos and set(pos_list[i:pos]) == {'.'}):
        return True
    else:
        return False

def pos_actions(pos_list, stacks):
    """Returns the list of possible actions and its cost in format (start, end, cost)."""
    actions = []
    for i,x in enumerate(pos_list):
        if x != '.' and visible_stacks(i, pos_list, stacks):
            nsteps = abs(i-types[x])+2 if len(stacks[types[x]]) == 0 else abs(i-types[x])+1
            actions.append((i, types[x], costs[x]*nsteps))
    for i in [2,4,6,8]:
        if set(stacks[i]) != set() and set(stacks[i]) != {types_rev[i]}:
            for j in visible_pos(i,pos_list):
                if stacks[i]:
                    nsteps = 2 + abs(j-i) if len(stacks[i]) == 1 else 1 + abs(j-i) 
                    actions.append((i, j, costs[stacks[i][-1]]*nsteps))
    return actions
            
def do_action(action, poslist, stacklist):
    """Returns new pos_list and stacks after action is performed"""
    stacks = copy.deepcopy(stacklist)
    pos_list = poslist.copy()
    if action[0] in [2,4,6,8]:
        x = stacks[action[0]].pop()
        pos_list[action[1]] = x
    else:
        x = pos_list[action[0]]
        pos_list[action[0]] = '.'
        stacks[action[1]].append(x)
    return pos_list, stacks

def do_path(path, poslist, stacklist):
    for action in path:
        poslist, stacklist = do_action(action, poslist, stacklist)
    return poslist, stacklist
    
def win(pos_list, stacks):
    """Says if a given position is winning"""
    return pos_list == ['.']*11 and stacks[2]==['A','A'] and stacks[4] == ['B','B'] and stacks[6] == ['C','C'] and stacks[8]==['D','D']

def find_min(pos_list, stacks):
    """Explore all possibilities using a DFS"""
    path_memory = dict()
    action_memory = dict()
    
    winning = []
    end = False
    path = []
    actions = pos_actions(pos_list, stacks)
    #print(actions)
    new_pos_list , new_stacks = pos_list, stacks
    cost = 0
    best_cost = 99999999999
    while not end:
        while actions:
            path.append(actions[0])
            pl = tuple(new_pos_list)
            st = tuple({x:tuple(y) for x,y in new_stacks.items()}.items())
            if (actions[0], pl, st) in action_memory:
                new_pos_list, new_stacks = action_memory[actions[0], pl, st]
            else:
                new_pos_list, new_stacks = do_action(actions[0], new_pos_list, new_stacks)
                action_memory[(actions[0], pl, st)] = (new_pos_list, new_stacks)
            path_memory[tuple(path)] = (new_pos_list, new_stacks)
            cost += actions[0][2]
            if cost>best_cost:
                break
            actions = pos_actions(new_pos_list, new_stacks)
        if win(new_pos_list, new_stacks):
            if cost < best_cost:
                best_cost = cost
                winning.append((path, cost))
                print('Win!', path, cost)
        #print(path, cost, best_cost)
        last = path.pop()
        cost -= last[-1]
        if len(path) <= 3:
            print("Cambio!", path, last)
            print(len(path_memory), len(action_memory))

        new_pos_list, new_stacks = path_memory[tuple(path)]
            
        actions = pos_actions(new_pos_list, new_stacks)
        actions = actions[actions.index(last)+1:]
        if not path and not actions:
            end = True
    return winning

find_min(pos_list, stacks)

Win! [(2, 0, 300), (2, 1, 300), (4, 3, 20), (4, 5, 3000), (3, 4, 30), (6, 10, 5), (8, 7, 2000), (8, 9, 30), (7, 8, 3000), (5, 8, 4000), (9, 4, 60), (10, 2, 10), (6, 3, 5), (3, 2, 2), (1, 6, 700), (0, 6, 700)] 14162
Win! [(2, 0, 300), (2, 1, 300), (4, 3, 20), (4, 5, 3000), (3, 4, 30), (8, 7, 2000), (8, 9, 30), (7, 8, 3000), (5, 8, 4000), (9, 4, 60), (6, 3, 4), (3, 2, 3), (6, 3, 5), (3, 2, 2), (1, 6, 700), (0, 6, 700)] 14154
Cambio! [(2, 0, 300), (2, 1, 300), (4, 3, 20)] (4, 5, 3000)
18723 489
Cambio! [(2, 0, 300), (2, 1, 300), (4, 3, 20)] (4, 7, 5000)
39504 598
Cambio! [(2, 0, 300), (2, 1, 300), (4, 3, 20)] (4, 9, 7000)
41245 733
Cambio! [(2, 0, 300), (2, 1, 300), (4, 3, 20)] (4, 10, 8000)
128311 1209
Cambio! [(2, 0, 300), (2, 1, 300), (4, 3, 20)] (6, 5, 2)
128346 1243
Cambio! [(2, 0, 300), (2, 1, 300), (4, 3, 20)] (6, 7, 2)
128379 1255
Cambio! [(2, 0, 300), (2, 1, 300), (4, 3, 20)] (6, 9, 4)
128415 1276
Cambio! [(2, 0, 300), (2, 1, 300), (4, 3, 20)] (6, 10, 5)
132762 1451
Cambio! [(2, 

Cambio! [(2, 0, 300), (2, 1, 300), (8, 3, 6000)] (8, 10, 40)
8134827 11919
Cambio! [(2, 0, 300), (2, 1, 300)] (8, 3, 6000)
8134827 11919
Cambio! [(2, 0, 300), (2, 1, 300), (8, 5, 4000)] (4, 3, 20)
8134931 11921
Cambio! [(2, 0, 300), (2, 1, 300), (8, 5, 4000)] (6, 7, 2)
8134939 11927
Cambio! [(2, 0, 300), (2, 1, 300), (8, 5, 4000)] (6, 9, 4)
8134947 11928
Cambio! [(2, 0, 300), (2, 1, 300), (8, 5, 4000)] (6, 10, 5)
8135670 11929
Cambio! [(2, 0, 300), (2, 1, 300), (8, 5, 4000)] (8, 7, 30)
8135672 11931
Cambio! [(2, 0, 300), (2, 1, 300), (8, 5, 4000)] (8, 9, 30)
8139426 11937
Cambio! [(2, 0, 300), (2, 1, 300), (8, 5, 4000)] (8, 10, 40)
8155956 11982
Cambio! [(2, 0, 300), (2, 1, 300)] (8, 5, 4000)
8155956 11982
Cambio! [(2, 0, 300), (2, 1, 300), (8, 7, 2000)] (4, 3, 20)
8190324 11984
Cambio! [(2, 0, 300), (2, 1, 300), (8, 7, 2000)] (4, 5, 20)
8190752 11985
Cambio! [(2, 0, 300), (2, 1, 300), (8, 7, 2000)] (6, 3, 4)
8284568 11986
Cambio! [(2, 0, 300), (2, 1, 300), (8, 7, 2000)] (6, 5, 2)
8355

9578428 12640
Cambio! [(2, 0, 300), (2, 3, 300), (8, 9, 2000)] (6, 5, 2)
9578432 12641
Cambio! [(2, 0, 300), (2, 3, 300), (8, 9, 2000)] (6, 7, 2)
9578435 12642
Cambio! [(2, 0, 300), (2, 3, 300), (8, 9, 2000)] (8, 5, 50)
9578445 12650
Cambio! [(2, 0, 300), (2, 3, 300), (8, 9, 2000)] (8, 7, 30)
9578453 12656
Cambio! [(2, 0, 300), (2, 3, 300)] (8, 9, 2000)
9578453 12656
Cambio! [(2, 0, 300), (2, 3, 300), (8, 10, 3000)] (4, 5, 20)
9578464 12658
Cambio! [(2, 0, 300), (2, 3, 300), (8, 10, 3000)] (4, 7, 40)
9578472 12659
Cambio! [(2, 0, 300), (2, 3, 300), (8, 10, 3000)] (4, 9, 60)
9578489 12673
Cambio! [(2, 0, 300), (2, 3, 300), (8, 10, 3000)] (6, 5, 2)
9578498 12674
Cambio! [(2, 0, 300), (2, 3, 300), (8, 10, 3000)] (6, 7, 2)
9578506 12675
Cambio! [(2, 0, 300), (2, 3, 300), (8, 10, 3000)] (6, 9, 4)
9594797 12689
Cambio! [(2, 0, 300), (2, 3, 300), (8, 10, 3000)] (8, 5, 50)
9594809 12694
Cambio! [(2, 0, 300), (2, 3, 300), (8, 10, 3000)] (8, 7, 30)
9594817 12698
Cambio! [(2, 0, 300), (2, 3, 300)

KeyboardInterrupt: 

# Part 2

In [143]:
import copy
from time import time
from random import randint

#Input
pos_list = ['.','.','.','.','.','.','.','.','.','.','.']
stacks = {2:['C','D','D','C'], 4:['D','B','C','B'], 6:['A','A','B','A'], 8:['B','C','A','D']}
costs = {'A':1, 'B':10, 'C':100, 'D':1000}
types = {'A':2,'B':4,'C':6,'D':8}
types_rev = {2:'A',4:'B',6:'C',8:'D'}

#Program
def visible_pos(nstack, pos_list):
    """Returns the list of possible positions from a given stack"""
    vis_pos=[]
    for i in range(len(pos_list)):
        if i<nstack and i not in [2,4,6,8] and set(pos_list[i:nstack+1])=={'.'}:
            vis_pos.append(i)
        if i>nstack and i not in [2,4,6,8] and set(pos_list[nstack:i+1]) == {'.'}:
            vis_pos.append(i)
    return vis_pos
    
def visible_stacks(pos, pos_list, stacks):
    """Says if amphipod at position pos can go to its stack"""
    x = pos_list[pos]
    i = types[x]
    if len(stacks[i])==4 or (len(stacks[i])>0 and set(stacks[i][0]) != {x}):
        return False
    if pos==i-1 or (pos<i and set(pos_list[pos+1:i]) == {'.'}):
        return True
    elif pos== i+1 or (i<pos and set(pos_list[i:pos]) == {'.'}):
        return True
    else:
        return False

def pos_actions(pos_list, stacks):
    """Returns the list of possible actions and its cost in format (start, end, cost)."""
    actions = []
    for i,x in enumerate(pos_list):
        if x != '.' and visible_stacks(i, pos_list, stacks):
            nsteps = abs(i-types[x])+(4-len(stacks[types[x]]))
            actions.append((i, types[x], costs[x]*nsteps))
    for i in [2,4,6,8]:
        if set(stacks[i]) != set() and set(stacks[i]) != {types_rev[i]}:
            for j in visible_pos(i,pos_list):
                if stacks[i]:
                    nsteps = 1 + abs(j-i) + (4-len(stacks[i])) 
                    actions.append((i, j, costs[stacks[i][-1]]*nsteps))
    return actions
            
def do_action(action, poslist, stacklist):
    """Returns new pos_list and stacks after action is performed"""
    stacks = copy.deepcopy(stacklist)
    pos_list = poslist.copy()
    if action[0] in [2,4,6,8]:
        x = stacks[action[0]].pop()
        pos_list[action[1]] = x
    else:
        x = pos_list[action[0]]
        pos_list[action[0]] = '.'
        stacks[action[1]].append(x)
    return pos_list, stacks

def do_path(path, poslist, stacklist):
    for action in path:
        poslist, stacklist = do_action(action, poslist, stacklist)
    return poslist, stacklist
    
def win(pos_list, stacks):
    """Says if a given position is winning"""
    return pos_list == ['.']*11 and stacks[2]==['A','A','A','A'] and stacks[4] == ['B','B','B','B'] \
        and stacks[6] == ['C','C','C','C'] and stacks[8]==['D','D','D','D']

def find_min(pos_list, stacks):
    """Explore all possibilities using a DFS"""
    path_memory = dict()
    action_memory = dict()
    
    winning = []
    end = False
    path = []
    actions = pos_actions(pos_list, stacks)
    #print(actions)
    new_pos_list , new_stacks = pos_list, stacks
    path_memory[tuple(path)] = (new_pos_list, new_stacks)
    cost = 0
    best_cost = 99999999999
    while not end:
        while actions:
            path.append(actions[0])
            pl = tuple(new_pos_list)
            st = tuple({x:tuple(y) for x,y in new_stacks.items()}.items())
            if (actions[0], pl, st) in action_memory:
                new_pos_list, new_stacks = action_memory[actions[0], pl, st]
            else:
                new_pos_list, new_stacks = do_action(actions[0], new_pos_list, new_stacks)
                action_memory[(actions[0], pl, st)] = (new_pos_list, new_stacks)
            path_memory[tuple(path)] = (new_pos_list, new_stacks)
            cost += actions[0][2]
            if cost>best_cost:
                break
            actions = pos_actions(new_pos_list, new_stacks)
        if win(new_pos_list, new_stacks):
            if cost < best_cost:
                best_cost = cost
                winning.append((path, cost))
                print('Win!', path, cost)
        #print(path, cost, best_cost)
        last = path.pop()
        cost -= last[-1]
        if len(path) <= 1:
            print("Cambio!", path, last)
            print(len(path_memory), len(action_memory))

        new_pos_list, new_stacks = path_memory[tuple(path)]
            
        actions = pos_actions(new_pos_list, new_stacks)
        actions = actions[actions.index(last)+1:]
        if not path and not actions:
            end = True
    return winning

find_min(pos_list, stacks)

Cambio! [(2, 0, 300)] (2, 1, 3000)
13264 5387
Cambio! [(2, 0, 300)] (2, 3, 3000)
16290 6115
Cambio! [(2, 0, 300)] (2, 5, 5000)
18325 6609
Cambio! [(2, 0, 300)] (2, 7, 7000)
20131 7098
Cambio! [(2, 0, 300)] (2, 9, 9000)
22021 7724
Cambio! [(2, 0, 300)] (2, 10, 10000)
72094 12734
Cambio! [(2, 0, 300)] (4, 1, 40)
94187 18301
Cambio! [(2, 0, 300)] (4, 3, 20)
97598 19051
Cambio! [(2, 0, 300)] (4, 5, 20)
99633 19444
Cambio! [(2, 0, 300)] (4, 7, 40)
102897 20174
Cambio! [(2, 0, 300)] (4, 9, 60)
106011 21029
Cambio! [(2, 0, 300)] (4, 10, 70)
132821 26660
Cambio! [(2, 0, 300)] (6, 1, 6)
147794 28830
Cambio! [(2, 0, 300)] (6, 3, 4)
150796 29245
Cambio! [(2, 0, 300)] (6, 5, 2)
152831 29409
Cambio! [(2, 0, 300)] (6, 7, 2)
154668 29569
Cambio! [(2, 0, 300)] (6, 9, 4)
156540 30024
Cambio! [(2, 0, 300)] (6, 10, 5)
180061 33059
Cambio! [(2, 0, 300)] (8, 1, 8000)
192361 34344
Cambio! [(2, 0, 300)] (8, 3, 6000)
195351 34593
Cambio! [(2, 0, 300)] (8, 5, 4000)
197386 34666
Cambio! [(2, 0, 300)] (8, 7, 200

Cambio! [(4, 1, 40)] (8, 7, 2000)
922343 125825
Cambio! [(4, 1, 40)] (8, 9, 2000)
922565 125865
Cambio! [(4, 1, 40)] (8, 10, 3000)
925285 126361
Cambio! [] (4, 1, 40)
925285 126361
Cambio! [(4, 3, 20)] (2, 0, 300)
928697 126363
Cambio! [(4, 3, 20)] (2, 1, 200)
929318 126364
Cambio! [(4, 3, 20)] (4, 5, 300)
929844 126462
Cambio! [(4, 3, 20)] (4, 7, 500)
930111 126504
Cambio! [(4, 3, 20)] (4, 9, 700)
930364 126556
Cambio! [(4, 3, 20)] (4, 10, 800)
933846 126803
Cambio! [(4, 3, 20)] (6, 5, 2)
934372 126903
Cambio! [(4, 3, 20)] (6, 7, 2)
934639 126934
Cambio! [(4, 3, 20)] (6, 9, 4)
934847 126977
Cambio! [(4, 3, 20)] (6, 10, 5)
936744 127192
Cambio! [(4, 3, 20)] (8, 5, 4000)
937270 127239
Cambio! [(4, 3, 20)] (8, 7, 2000)
937537 127259
Cambio! [(4, 3, 20)] (8, 9, 2000)
937745 127278
Cambio! [(4, 3, 20)] (8, 10, 3000)
939642 127413
Cambio! [] (4, 3, 20)
939642 127413
Cambio! [(4, 5, 20)] (2, 0, 300)
941678 127415
Cambio! [(4, 5, 20)] (2, 1, 200)
941949 127416
Cambio! [(4, 5, 20)] (2, 3, 200)

Cambio! [(6, 5, 2)] (6, 7, 30)
2864791 190533
Cambio! [(6, 5, 2)] (6, 9, 50)
2865062 190554
Cambio! [(6, 5, 2)] (6, 10, 60)
2867097 190625
Cambio! [(6, 5, 2)] (8, 7, 2000)
2867623 190653
Cambio! [(6, 5, 2)] (8, 9, 2000)
2867894 190668
Cambio! [(6, 5, 2)] (8, 10, 3000)
2869929 190726
Cambio! [] (6, 5, 2)
2869929 190726
Cambio! [(6, 7, 2)] (2, 0, 300)
2871767 190728
Cambio! [(6, 7, 2)] (2, 1, 200)
2871975 190729
Cambio! [(6, 7, 2)] (2, 3, 200)
2872242 190730
Cambio! [(6, 7, 2)] (2, 5, 400)
2872768 190731
Cambio! [(6, 7, 2)] (4, 0, 50)
2874821 190732
Cambio! [(6, 7, 2)] (4, 1, 40)
2875029 190733
Cambio! [(6, 7, 2)] (4, 3, 20)
2875296 190734
Cambio! [(6, 7, 2)] (4, 5, 20)
2875978 190735
Cambio! [(6, 7, 2)] (6, 0, 80)
2877796 191064
Cambio! [(6, 7, 2)] (6, 1, 70)
2878004 191148
Cambio! [(6, 7, 2)] (6, 3, 50)
2878271 191179
Cambio! [(6, 7, 2)] (6, 5, 30)
2878797 191207
Cambio! [(6, 7, 2)] (8, 9, 2000)
2879380 191220
Cambio! [(6, 7, 2)] (8, 10, 3000)
2882483 191246
Cambio! [] (6, 7, 2)
288248

Cambio! [(8, 9, 2000)] (8, 0, 10)
3468420 226863
Cambio! [(8, 9, 2000)] (8, 1, 9)
3468650 227001
Cambio! [(8, 9, 2000)] (8, 3, 7)
3468877 227039
Cambio! [(8, 9, 2000)] (8, 5, 5)
3469148 227054
Cambio! [(8, 9, 2000)] (8, 7, 3)
3469728 227067
Cambio! [] (8, 9, 2000)
3469728 227067
Cambio! [(8, 10, 3000)] (2, 0, 300)
3491159 227069
Cambio! [(8, 10, 3000)] (2, 1, 200)
3492976 227070
Cambio! [(8, 10, 3000)] (2, 3, 200)
3494813 227071
Cambio! [(8, 10, 3000)] (2, 5, 400)
3496848 227072
Cambio! [(8, 10, 3000)] (2, 7, 600)
3499917 227073
Cambio! [(8, 10, 3000)] (2, 9, 800)
3513135 229716
Cambio! [(8, 10, 3000)] (4, 0, 50)
3539721 229717
Cambio! [(8, 10, 3000)] (4, 1, 40)
3542441 229718
Cambio! [(8, 10, 3000)] (4, 3, 20)
3544338 229719
Cambio! [(8, 10, 3000)] (4, 5, 20)
3547041 229720
Cambio! [(8, 10, 3000)] (4, 7, 40)
3557285 229721
Cambio! [(8, 10, 3000)] (4, 9, 60)
3583776 232220
Cambio! [(8, 10, 3000)] (6, 0, 7)
3607789 232221
Cambio! [(8, 10, 3000)] (6, 1, 6)
3609604 232222
Cambio! [(8, 10,

[([], 57002), ([], 56982)]